`Rasmus Utz Faber`

`Martín Omil Nogales`

# Unit 3 - Part 2: Trained CNNs



## 1. Selection of models, data extraction and preprocessing

In [ ]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np

# Step 1: Load and Preprocess Data
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Extract a validation set from the training set (10%)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

IMG_SIZE = (224, 224)

def resize_images(images):
    resized_images = []
    for image in images:
        resized_image = tf.image.resize(image, IMG_SIZE)  # Keep as Tensor
        resized_images.append(resized_image)
    return tf.convert_to_tensor(resized_images)  # Convert the entire list to a tensor

x_train = resize_images(x_train)
x_val = resize_images(x_val)
x_test = resize_images(x_test)

# Data Augmentation Layer
data_augmentation = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomZoom(0.2),
    keras.layers.RandomTranslation(0.1, 0.1)
])

## 2. Feature Extraction

In [ ]:
# Step 2: Feature Extraction (Initial Setup)
def build_feature_extractor(model_name):
    base_model = None

    if model_name == 'ResNet50V2':
        base_model = keras.applications.ResNet50V2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'VGG19':
        base_model = keras.applications.VGG19(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'MobileNet':
        base_model = keras.applications.MobileNet(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'InceptionV3':
        base_model = keras.applications.InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    elif model_name == 'ResNet101V2':
        base_model = keras.applications.ResNet101V2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))

    base_model.trainable = False  # Freeze the pretrained model

    model = keras.Sequential([
        data_augmentation,  # Applying Data Augmentation
        base_model,
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(100, activation='softmax')  # CIFAR-100 has 100 classes
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [ ]:
# Step 2: Training the models for Feature Extraction

def train_model(model_name):
    model = build_feature_extractor(model_name)
    print(f"\nTraining {model_name} with Feature Extraction...")

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=5,
        batch_size=32
    )
    
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"{model_name} Test Accuracy: {test_acc * 100:.2f}%")

    return history, test_acc


# Models to train
model_names = ['ResNet50V2', 'VGG19', 'MobileNet', 'InceptionV3', 'ResNet101V2']

results = {}

for model_name in model_names:
    history, test_acc = train_model(model_name)
    results[model_name] = test_acc

## 3. Fine Tuning

In [ ]:
# Step 3: Fine-Tuning the models

def fine_tune_model(model_name):
    model = keras.models.load_model(f'{model_name}_feature_extractor.h5')

    # Unfreeze some layers (last 2 blocks)
    base_model = model.layers[0]
    base_model.trainable = True

    for layer in base_model.layers[:-30]:  # Keep most layers frozen
        layer.trainable = False

    # Compile the model with a low learning rate
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    print(f"\nFine-tuning {model_name}...")

    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=5,
        batch_size=32
    )

    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"{model_name} Fine-Tuning Test Accuracy: {test_acc * 100:.2f}%")

    return history, test_acc



## 4. Training

In [ ]:
# Models to train and fine-tune
model_names = ['ResNet50V2', 'VGG19', 'MobileNet', 'InceptionV3', 'ResNet101V2']

results = {}

for model_name in model_names:
    # Feature Extraction
    history, test_acc = train_model(model_name)
    results[f'{model_name} (Feature Extraction)'] = test_acc

    # Fine-Tuning
    history, test_acc = fine_tune_model(model_name)
    results[f'{model_name} (Fine-Tuning)'] = test_acc

print("\nAll Results:", results)


## 5. Graph

In [ ]:
import matplotlib as plt

# Generate Performance Comparison Graph
labels = list(results.keys())
scores = list(results.values())

plt.figure(figsize=(12, 6))
plt.barh(labels, scores, color='skyblue')
plt.xlabel('Test Accuracy')
plt.title('Performance Comparison of Pretrained Models on CIFAR-100')
plt.grid(True)
plt.show()


In [ ]:
import pandas as pd

# Generate Performance Comparison Table
results_df = pd.DataFrame(list(results.items()), columns=['Model and Strategy', 'Test Accuracy'])

print("\nPerformance Comparison Table:\n")
print(results_df.to_string(index=False))